# Strategy

Prior to diving into the strategy itself, we will gather the required data. I am using the Coingecko API as my Velodata access is limited and I am not able to get the required 365 days historic data.

In [23]:
import polars as pl
import pycoingecko

cg = pycoingecko.CoinGeckoAPI()

coin_list = ['aptos', 'arbitrum', 'avalanche-2', 'binancecoin', 'bitcoin-cash', 'bitcoin', 'bonk', 'cardano', 'celestia', 'chainlink', 'cosmos', 'dogecoin', 'dogwifcoin', 'ethereum', 'floki', 'injective-protocol', 'litecoin', 'matic-network', 'near', 'notcoin', 'optimism', 'pepe', 'polkadot', 'ripple', 'shiba-inu', 'solana', 'sui', 'the-open-network', 'thorchain', 'tron']

df_prices = pl.DataFrame()

for coin in coin_list:
    data = cg.get_coin_market_chart_by_id(coin, 'usd', 365)
    df_coin = (pl.DataFrame(data['prices'], strict=False)
               .transpose()
               .with_columns(pl.lit(coin).alias('ticker'), 
                             pl.all().cast(pl.String))
               )
    df_coin.write_csv(f"./data/{coin}.csv")

Next, we combine and clean up the dataframe.

In [43]:
try:
    df = (
        pl.scan_csv(f"./data/*.csv", infer_schema_length=0).with_columns(
            pl.from_epoch(pl.col('column_0').cast(pl.Float32), time_unit='ms').alias('timestamp'),
            pl.col('column_1').cast(pl.Float32).alias('price')
        ).drop(
            ['column_0', 'column_1']
        )
    )
except pl.exceptions.ComputeError as e:
    raise(e)

df = df.collect()

# round the timestamp
df.with_columns(
    pl.col('timestamp').dt.date()
).sort(['timestamp'])

ticker,timestamp,price
str,date,f32
"""aptos""",2023-08-11,7.09712
"""arbitrum""",2023-08-11,1.157507
"""avalanche-2""",2023-08-11,12.440173
"""binancecoin""",2023-08-11,239.595825
"""bitcoin-cash""",2023-08-11,228.957672
…,…,…
"""the-open-network""",2024-08-10,6.665984
"""thorchain""",2024-08-10,3.23603
"""thorchain""",2024-08-10,3.230335
